In [1]:
import pandas as pd
import requests
import os
import ast
import sys
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append("/home/jovyan/workdir/document_classification/utils")
from generic import get_random_sample,ast_df


excel_file = "/home/jovyan/workdir/document_classification/data/DataSet.xlsx"
excel_sheets = pd.read_excel(excel_file, sheet_name=None)
for sheet_name, df in excel_sheets.items():
    print(sheet_name)
    globals()[sheet_name] = df

train_data
test_data


In [2]:
def prepare_filename_index(df):
    df = df.copy()
    df["doc_index"] = df.index
    df["filename"] = "doc_" + df["doc_index"].astype(str) + ".pdf"
    df.drop(columns=["doc_index"], inplace=True)
    return df


train_data = prepare_filename_index(train_data)
test_data = prepare_filename_index(test_data)

In [3]:
train_extracted_df = pd.read_csv('/home/jovyan/workdir/document_classification/data/train/pdf_extraction_pypdf2_results_train.csv')
train_extracted_df = train_extracted_df[train_extracted_df["status"] == "Success"]
test_extracted_df = pd.read_csv('/home/jovyan/workdir/document_classification/data/test/pdf_extraction_pypdf2_results_test.csv')
test_extracted_df = test_extracted_df[test_extracted_df["status"] == "Success"]


In [4]:
ast_df(train_extracted_df)
ast_df(test_extracted_df)

Note: ast.literal_eval not needed or failed: malformed node or string on line 1: <ast.Attribute object at 0x7fc85d206110>
Note: ast.literal_eval not needed or failed: malformed node or string on line 1: <ast.Attribute object at 0x7fc85d206d70>


In [5]:
train_extracted_df = train_extracted_df.dropna()
test_extracted_df = test_extracted_df.dropna()

train_extracted_df.info()
test_extracted_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1279 entries, 1 to 1651
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   filename   1279 non-null   object 
 1   num_pages  1279 non-null   float64
 2   text       1279 non-null   object 
 3   status     1279 non-null   object 
dtypes: float64(1), object(3)
memory usage: 50.0+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 217 entries, 1 to 255
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   filename   217 non-null    object 
 1   num_pages  217 non-null    float64
 2   text       217 non-null    object 
 3   status     217 non-null    object 
dtypes: float64(1), object(3)
memory usage: 8.5+ KB


In [6]:
train_data

,datasheet_link,target_col,filename
0,https://lfillumination.com/files/specsheets/EF...,lighting,doc_0.pdf
1,https://lfillumination.com/files/specsheets/EF...,lighting,doc_1.pdf
2,https://lfillumination.com/files/specsheets/EF...,lighting,doc_2.pdf
3,https://www.waclighting.com/storage/waclightin...,lighting,doc_3.pdf
4,https://www.acuitybrands.com/api/products/geta...,lighting,doc_4.pdf
...,...,...,...
2565,https://www.memoryprotectiondevices.com/datash...,others,doc_2565.pdf
2566,https://www.memoryprotectiondevices.com/datash...,others,doc_2566.pdf
2567,https://www.memoryprotectiondevices.com/datash...,others,doc_2567.pdf
2568,http://www.switchcomp.com/switchcomp_ecommerce...,others,doc_2568.pdf


In [7]:
test_data

,datasheet_link,target_col,filename
0,https://lumenart.com/images/alume/awl-01_specs...,lighting,doc_0.pdf
1,https://lumenart.com/images/fabric/rdc/rdc_spe...,lighting,doc_1.pdf
2,https://lumenart.com/images/fabric/cyp/cyp_spe...,lighting,doc_2.pdf
3,https://lumenart.com/images/designer/wlp_specs...,lighting,doc_3.pdf
4,https://lumenart.com/images/designer/wcp/wcp-s...,lighting,doc_4.pdf
...,...,...,...
395,https://catalog.belden.com/techdata/EN/3092A_t...,cable,doc_395.pdf
396,https://html5.dcatalog.com/?docid=8e9daddd-82b...,cable,doc_396.pdf
397,https://timesmicrowave.com/wp-content/uploads/...,cable,doc_397.pdf
398,https://catalog.belden.com/techdata/EN/1505A_t...,cable,doc_398.pdf


In [8]:
train_df = pd.merge(train_data, train_extracted_df, on="filename", how="inner")
test_df = pd.merge(test_data, test_extracted_df, on="filename", how="inner")
train_df = train_df[["filename", "text", "num_pages", "target_col"]]
test_df = test_df[["filename", "text", "num_pages", "target_col"]]

In [9]:
train_df

,filename,text,num_pages,target_col
0,doc_0.pdf,EF400 System # EF408B\nDIE CAST CYLINDRICAL L...,1.0,lighting
1,doc_2.pdf,EF400 System # EF407B\nDIE CAST CYLINDRICAL L...,1.0,lighting
2,doc_3.pdf,ADJ UST ABLE BEAM WALL WASH 12V\n5221\nORDER...,4.0,lighting
3,doc_4.pdf,Type:\nProject:HDMC\nSurface Mount with FAR-UV...,11.0,lighting
4,doc_6.pdf,Extruded Aluminum\nHousing\nSatin Acrylic\nDif...,1.0,lighting
...,...,...,...,...
1274,doc_2511.pdf,EM ELECTRET \nCONDENSER \nMICROPHONE \nAcousti...,8.0,others
1275,doc_2536.pdf,FLYER // AUTOMOTIVE POWER SUPPLY // ORDER CODE...,1.0,others
1276,doc_2556.pdf,"GlobTek, Inc.\nwww.globtek.com\n186 Veterans D...",20.0,others
1277,doc_2568.pdf,Mechanical Dimensions: Inches [mm]\nMounting H...,2.0,others


In [10]:
print("Dataset Info:")
print("-" * 50)
print(train_df.info())
print("\nMissing Values:")
print("-" * 50)
print(train_df.isnull().sum())


Dataset Info:
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279 entries, 0 to 1278
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   filename    1279 non-null   object 
 1   text        1279 non-null   object 
 2   num_pages   1279 non-null   float64
 3   target_col  1279 non-null   object 
dtypes: float64(1), object(3)
memory usage: 40.1+ KB
None

Missing Values:
--------------------------------------------------
filename      0
text          0
num_pages     0
target_col    0
dtype: int64


In [11]:

print("\nTrain: Target Class Distribution:")
print("-" * 50)
target_dist = train_df['target_col'].value_counts()
print(target_dist)
print("\nTrain: Percentage Distribution:")
print(target_dist/len(train_df)*100)

print("\nTest: Target Class Distribution:")
print("-" * 50)
target_dist = test_df['target_col'].value_counts()
print(target_dist)
print("\nTest: Percentage Distribution:")
print(target_dist/len(test_df)*100)



Train: Target Class Distribution:
--------------------------------------------------
target_col
fuses       495
lighting    325
cable       310
others      149
Name: count, dtype: int64

Train: Percentage Distribution:
target_col
fuses       38.702111
lighting    25.410477
cable       24.237686
others      11.649726
Name: count, dtype: float64

Test: Target Class Distribution:
--------------------------------------------------
target_col
lighting    65
cable       64
others      49
fuses       39
Name: count, dtype: int64

Test: Percentage Distribution:
target_col
lighting    29.953917
cable       29.493088
others      22.580645
fuses       17.972350
Name: count, dtype: float64


In [12]:
text, target = get_random_sample(train_df,class_name="others")

Random Document Sample:
--------------------------------------------------

Text Content:
--------------------------------------------------
US+1-847-639-6400   sales@coilcraft.com
+44-1236-730595   sales@coilcraft-eur ope.com UK
+886-2-2264 3646   sales@coilcraft.com.tw Taiwan
+86-21-6218 8074   sales@coilcraft.com.cn China
+ 65-6484 8412   sales@coilcraft.com.sg Singapore© Coilcraft Inc. 2023
This pr oduct may not be used in medical or high
risk applications without prior Coilcraft appr oval.
Specification subject to change without notice.
Please check web site for latest infor mation.AEC
Q200125°C +Halogen
FreeSample
& buySuppor t
& FA QWeb
pageTools &
softw are
Shielded Power Inductors  – XAL50 xxDocument 908-1
 
Document 908-1 Revised 06/23/23Designer’s Kit C445  contains 3 of each value 
Core material  Composite
Environmental  RoHS compliant, halogen free
Terminations  RoHS compliant tin-silver (96.5/3.5) over copper  
Other terminations available at additional cost.
Weight XAL50

In [13]:
train_df['text_length'] = train_df['text'].str.len()
print("\nText Length Statistics:")
print("-" * 50)
print(train_df['text_length'].describe())
print("-" * 50)
print(f"90th percentile: {train_df['text_length'].quantile(0.90):.0f}")
print(f"95th percentile: {train_df['text_length'].quantile(0.95):.0f}")
print(f"99th percentile: {train_df['text_length'].quantile(0.99):.0f}")


Text Length Statistics:
--------------------------------------------------
count      1279.000000
mean       8116.773260
std       10393.739254
min          17.000000
25%        4189.000000
50%        6560.000000
75%        8672.000000
max      291390.000000
Name: text_length, dtype: float64
--------------------------------------------------
90th percentile: 14585
95th percentile: 23076
99th percentile: 37138


In [14]:
print("\nNumber of Pages Statistics:")
print("-" * 50)
print(train_df['num_pages'].describe())
print("-" * 50)
print(f"90th percentile: {train_df['num_pages'].quantile(0.90):.0f}")
print(f"95th percentile: {train_df['num_pages'].quantile(0.95):.0f}")
print(f"99th percentile: {train_df['num_pages'].quantile(0.99):.0f}")


Number of Pages Statistics:
--------------------------------------------------
count    1279.000000
mean        3.971853
std         3.876112
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max        98.000000
Name: num_pages, dtype: float64
--------------------------------------------------
90th percentile: 7
95th percentile: 10
99th percentile: 16


In [15]:
train_df.to_csv('/home/jovyan/workdir/document_classification/data/train/uncleaned_train.csv', index=False)
test_df.to_csv('/home/jovyan/workdir/document_classification/data/test/uncleaned_test.csv', index=False)


In [17]:
# Create a dictionary containing both dataframes
data_dict = {"train": train_df, "test": test_df}
# Save as pickle
pd.to_pickle(
    data_dict, "/home/jovyan/workdir/document_classification/data/v0/combined_data.pkl"
)
